In [211]:
import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import os
import numpy as np
import pandas as pd
import wandb
import plotly.express as px

# Pytorch implementation

In [212]:
use_wandb = False

In [213]:
config = {
    "learning_rate": 1e-3,
    "dataset": "Titanic",
    "epochs": 500,
    "batch_size": 16,
    "loss": "binary cross-entropy",
    "optimizer": "SGD"
}

In [214]:
if use_wandb:
    wandb.login()
    wandb.init(project="assignment-1", name="pytorch", reinit=True, config=config)
    # wandb.config = config

In [215]:
# device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
device = 'cpu'
print(f"Using {device} device")

Using cpu device


## Loading the dataset

In [216]:
class TitanicDataset(Dataset):
    def __init__(self, data_dir):
        self.labels = pd.read_csv(os.path.join(data_dir, 'labels.csv')).to_numpy(dtype='float32')
        self.data = pd.read_csv(os.path.join(data_dir, 'data.csv')).to_numpy(dtype='float32')

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        data = torch.from_numpy(self.data[idx])
        label = torch.from_numpy(self.labels[idx])
        # data = self.data[idx]
        # label = self.labels[idx]

        return data, label

In [217]:
train_data = TitanicDataset(data_dir='data/train/')
val_data = TitanicDataset(data_dir='data/val/')
test_data = TitanicDataset(data_dir='data/test/')

In [218]:
features, label = train_data.__getitem__(idx=0)
print(features, label)

tensor([2.0000, 0.3322, 1.0000, 1.0000, 3.5115]) tensor([1.])


In [219]:
train_dataloader = DataLoader(train_data, batch_size=config['batch_size'], shuffle=True)
val_dataloader = DataLoader(val_data, batch_size=config['batch_size'], shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=config['batch_size'], shuffle=True)

In [220]:
for batch, (x, y) in enumerate(train_dataloader):
    print(batch, x.shape, y.shape)
    break

0 torch.Size([16, 5]) torch.Size([16, 1])


## Defining the model

In [221]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Sequential(
            nn.Linear(5, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        out = self.linear(x)
        return out

In [222]:
model = NeuralNetwork().to(device)
print(model)

if use_wandb: 
    wandb.config['model'] = model.__dict__['_modules']['linear']

NeuralNetwork(
  (linear): Sequential(
    (0): Linear(in_features=5, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=1, bias=True)
    (5): Sigmoid()
  )
)


## Training

In [223]:
loss_fn = nn.BCELoss()
if config['optimizer'] == 'Adam':
    optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'])
elif config['optimizer'] == 'SGD':
    optimizer = torch.optim.SGD(model.parameters(), lr=config['learning_rate'])

In [224]:
# wandb.watch(model, loss_fn, log='all', log_freq=10)

In [225]:
def calculate_acc(y_pred, y) -> float:
    # print(y_pred)
    y_pred = (y_pred>0.5)
    # print(y_pred)
    # print(y)

    # print(f'{((y == y_pred).sum().item())}/{len(y)}={((y == y_pred).sum().item())/len(y)}')
    return ((y == y_pred).sum().item())/len(y)

In [226]:
loss_graph = []
acc_graph = []

def train(dataloader, model, loss_fn, optimizer, epoch):
    model.train()
    avg_loss, avg_acc = 0, 0
    # curr = 0
    for x, y in dataloader:
        # curr += len(y)
        x, y = x.to(device), y.to(device)

        # Get prediction and compute loss
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        avg_loss += loss.item()

        # Update parameters
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        avg_acc += calculate_acc(y_pred, y)
        # print(f'loss: {loss.item():>5f} acc: {100*correct:>5f} | {curr}/{len(dataloader.dataset)}')

    avg_loss /= len(dataloader)
    avg_acc /= len(dataloader)
    
    loss_graph.append(avg_loss)
    acc_graph.append(avg_acc)
    
    if (epoch % 50 == 0):
        print(f"Epoch {epoch}")
        print(f'avg loss: {avg_loss:>5f} avg acc: {avg_acc:>5f}')
    
    if use_wandb:
        wandb.log({'loss': avg_loss})
        wandb.log({'acc': avg_acc})

In [227]:
val_loss_graph = []
val_acc_graph = []

def val(dataloader, model, loss_fn, epoch):    
    model.eval()
    avg_loss, avg_acc = 0, 0
    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)
            y_pred = model(x)
            avg_loss += loss_fn(y_pred, y).item()
            avg_acc += calculate_acc(y_pred, y)
    
    avg_loss /= len(dataloader)
    avg_acc /= len(dataloader)

    val_loss_graph.append(avg_loss)
    val_acc_graph.append(avg_acc)
    
    if (epoch % 50 == 0):
        print(f'avg val loss: {avg_loss:>5f} avg val acc: {avg_acc:>5f}')
        print('-------------------------------')
    
    if use_wandb:
        wandb.log({'val_loss': avg_loss})
        wandb.log({'val_acc': avg_acc})

In [228]:
epochs = config['epochs']
for epoch in range(epochs):
    # print(f"Epoch {epoch+1}")
    train(train_dataloader, model, loss_fn, optimizer, epoch)
    val(val_dataloader, model, loss_fn, epoch)
    # print('-------------------------------')

Epoch 0
avg loss: 0.692685 avg acc: 0.452635
avg val loss: 0.691061 avg val acc: 0.515114
-------------------------------
Epoch 50
avg loss: 0.631553 avg acc: 0.628917
avg val loss: 0.640330 avg val acc: 0.621732
-------------------------------
Epoch 100
avg loss: 0.591981 avg acc: 0.701033
avg val loss: 0.607248 avg val acc: 0.710376
-------------------------------
Epoch 150
avg loss: 0.574523 avg acc: 0.708511
avg val loss: 0.592192 avg val acc: 0.725082
-------------------------------
Epoch 200
avg loss: 0.561148 avg acc: 0.718661
avg val loss: 0.592883 avg val acc: 0.719363
-------------------------------
Epoch 250
avg loss: 0.554403 avg acc: 0.717058
avg val loss: 0.587239 avg val acc: 0.721405
-------------------------------
Epoch 300
avg loss: 0.552915 avg acc: 0.724181
avg val loss: 0.585398 avg val acc: 0.717729
-------------------------------
Epoch 350
avg loss: 0.551025 avg acc: 0.722222
avg val loss: 0.599176 avg val acc: 0.702614
-------------------------------
Epoch 400
a

In [229]:
if use_wandb:
    wandb.finish()

In [230]:
fig = px.line({'loss': loss_graph, 'val_loss': val_loss_graph})
fig.show()

In [231]:
fig = px.line({'acc': acc_graph, 'val_acc': val_acc_graph})
fig.show()